In [8]:
GENIUS_API_TOKEN='F2SBCBsZrWPf0Isy6wKwnYyVDpZF9z67Zzd_UjHPNvEylkr0j6D0Qo0sQMeCGfW9'

In [21]:
import requests
from bs4 import BeautifulSoup
import os
import re
import pickle
import numpy as np

In [15]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
request_song_url('Lana Del Rey', 2)

Found 2 songs by Lana Del Rey


['https://genius.com/Lana-del-rey-young-and-beautiful-lyrics',
 'https://genius.com/Lana-del-rey-summertime-sadness-lyrics']

In [11]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    # lyrics = html.find('div', {'class': "lyrics"}).get_text()
    for tag in html.select('div[class^="Lyrics__Container"], .song_body-lyrics p'):
        lyrics = tag.get_text(strip=True, separator='\n')
        if lyrics:
            print(lyrics)
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics
# DEMO
print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

[Verse 1]
I've seen the world, done it all
Had my cake now
Diamonds, brilliant,
and Bel Air now
Hot summer nights, mid-July
When you and I were forever wild
The crazy days, city lights
The way you'd play with me like a child
[Chorus]
Will you still love me when I'm no longer young and beautiful?
Will you still love me when I got nothing but my aching soul?
I know you will, I know you will, I know that you will
Will you still love me when I'm no longer beautiful?
[Verse 2]
I've seen the world, lit it up as my stage now
Channeling angels in the new age now
Hot summer days
, rock and roll
The way you'd play for me at your show
And all the ways I got to know
Your pretty face and electric soul
[Chorus]
Will you still love me when I'm no longer young and beautiful?
Will you still love me when I got nothing but my aching soul?
I know you will, I know you will, I know that you will
Will you still love me when I'm no longer beautiful?
[Bridge]
Dear Lord, when I get to heaven
Please, let me brin

In [57]:
lyrs = []
def write_lyrics_to_file(artist_name, song_count):
    try:
        f = open('lyrics/' + artist_name.lower() + '.txt', 'wb')
    except:
        print('e')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        lyrics = scrape_song_lyrics(url)
        lyrs.append(lyrics.encode("utf8"))
        f.write(lyrics.encode("utf8"))
    f.close()
    num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.txt', 'rb'))
    print('Wrote {} lines to file from {} songs'.format(num_lines, song_count))

In [58]:
write_lyrics_to_file('Kendrick Lamar', 1)

Found 1 songs by Kendrick Lamar
[Intro]
Nobody pray for me
It been that day for me
Way (Yeah, yeah)
[Verse 1]
Ayy, I remember syrup sandwiches and crime allowances
Finesse a nigga with some counterfeits, but now I'm countin’ this
Parmesan where my accountant lives, in fact I'm downin' this
D’USSÉ with my boo bae tastes like Kool-Aid for the analysts
Girl, I can buy your ass the world with my paystub
Ooh, that pussy good, won't you sit it on my taste bloods?
I get way too petty once you let me do the extras
Pull up on your block, then break it down: we playin' Tetris
AM to the PM, PM to the AM, funk
Piss out your per diem, you just gotta hate 'em, funk
If I quit your BM, I still ride Mercedes, funk
If I quit this season, I still be the greatest, funk
My left stroke just went viral
Right stroke put lil' baby in a spiral
Soprano C, we like to keep it on a high note
It's levels to it, you and I know
[Chorus]
Bitch, be humble (Hol' up, bitch)
Sit down (Hol' up, lil’, hol’ up, lil' bitch)
Be

In [59]:
lyrs

[b"Who that nigga thinkin' that he frontin' on Man-Man? \nGet the fuck off my stage, I'm the Sandman \nGet the fuck off my dick, that ain't right\nI make a play fucking up your whole life\nI'm so fuckin' sick and tired of the Photoshop\nShow me somethin' natural like afro on Richard Pryor\nShow me somethin' natural like ass with some stretch marks\nStill will take you down right on your mama's couch in Polo socks\nAyy, this shit way too crazy, ayy, you do not amaze me, ayy\nI blew cool from AC, ayy, Obama just paged me, ayy\nI don't fabricate it, ayy, most of y'all be fakin', ayy\nI stay modest 'bout it, ayy, she elaborate it, ayy\nThis that Grey Poupon, that Evian, that TED Talk, ayy\nWatch my soul speak, you let the meds talk, ayy\nIf I kill a nigga, it won't be the alcohol, ayy\nI'm the realest nigga after all\nBitch, be humble \nSit down \nBe humble \nSit down \nBe humble \nBitch, sit down \nBe humble \nSit down \nBe humble \nSit down \nBe humble \nSit down \nBe humble \nBitch, sit

In [60]:
infile = open('namelist-from-genius.pkl','rb')
artist_list = pickle.load(infile)
infile.close()

In [61]:
artist_list_np = np.array(artist_list)
artist_list_np

array(['A1 x J', 'A2', 'A2', ..., '9th Stree', '9th Wonde',
       '9 to 5 Original Broadway Ca'], dtype='<U225')

In [ ]:
for name in artist_list_np:
    print(name)
    write_lyrics_to_file(str(name), 1)

A1 x J
